# Caller

`Caller` is a class that makes it easy to call code in different threads/tasks. 

One caller instance is created per thread and can start children callers in new threads by name or using a pool of workers with the 'to_thread' method.

## Pending

If a method call returns an `async_kernel.Pending` the result of the call can be obtained by awaiting the pending.

## Usage by the kernel

The kernel uses two `Caller` instances; one each for `shell` and `control`. The shell thread event loop is normally the `MainThread`, 
but could be any thread depending on where the kernel was started. The control event loop is always named `ControlThread` and is a child of the shell caller. The
shell caller is accessible at `kernel.caller` while the kerenel is running.

In [ ]:
# A magic method provided by async kernel
%callers

In [ ]:
from async_kernel import Caller

Caller()

## Example
**This example requires ipywidgets!**

In [ ]:
import random
import time

import ipywidgets as ipw

outputs = {}
stop = ipw.RadioButtons(value=None, options=["Stop"])
display(stop)


def my_func(n):
    caller = Caller()
    if not (out := outputs.get(caller)):
        outputs[caller] = out = ipw.HTML(description=str(caller), style={"description_width": "initial"})
        display(out)
    sleep_time = random.random() / 4
    out.value = f"{n=:04d} sleeping {sleep_time * 1000:03.0f} ms"
    time.sleep(sleep_time)
    return n


async def run_forever():
    n = 0
    while not stop.value:
        n += 1
        yield Caller().to_thread(my_func, n)


async for fut in Caller().as_completed(run_forever()):
    result = await fut
    print(f"Finished: {result}", end="\r")

In [ ]:
%callers